## 서울 지하철 데이터로 나만의 지도 만들기

In [ ]:
import pandas as pd
import requests
import time

#키 값 변수
key = "934DA1A2-8005-39B2-85FC-BD69BC2259C9"
csv = "서울교통공사_역주소 및 전화번호_20250318.csv"


# VWorld Geocoding API 엔드포인트
url = "https://api.vworld.kr/req/address"

# requests.Session 생성 및 User-Agent 설정
with requests.Session() as session:
    # 세션 객체에서 웹브라우저 정보 (user agent) 주입
    session.headers.update({
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"
    })
    
    # CSV 파일 읽기
    try:
        df = pd.read_csv(csv, encoding='utf-8')
    except:
        # euc-kr로 돌아가게 예외처리
        df = pd.read_csv(csv, encoding='euc-kr')
    
    # 주소 컬럼 찾기
    address_col = None
    for col in df.columns:
        if '주소' in col or 'address' in col.lower() or '소재지' in col:
            address_col = col
            break
    
    # 위도, 경도 컬럼 추가
    df['latitude'] = None
    df['longitude'] = None
    
    # 각 행에 대해 API 호출
    for idx, row in df.iterrows():
        address = row[address_col]
        
        # API 요청 파라미터 설정
        params = {
            "service": "address",
            "request": "getcoord",
            "crs": "epsg:4326",
            "address": address,
            "format": "json",
            "type": "road",
            "key": key
        }
        
        # API 호출
        r = session.get(url, params=params)
        r.encoding = "utf-8"
            
        data = r.json()


# 엑셀 파일로 저장
df.to_excel("서울시_지하철역_좌표_완성.xlsx")

KeyboardInterrupt: 